In [32]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
from scipy.optimize import curve_fit
import glob
import os
import sys
import matplotlib.backends.backend_pdf
import matplotlib.gridspec as gridspec
from scipy.stats import norm
from scipy.stats import lognorm
from scipy import linalg
import pprint
import re
plt.rc('font', family='serif')
from matplotlib import rc
import pandas as pd
from astropy import constants as const
from astropy import units as units
from IPython.display import Markdown as md
import sympy
import pandas
from sympy import integrate, diff, sqrt, cos, sin, pi, exp, log
from sympy.abc import * 
i = sqrt(-1)
import numpy as np
import sympy.printing as printing
latp = printing.latex
hbar = sympy.symbols("hbar", real=True)
import mpmath 
import plotly.express as px
import plotly.graph_objs as go
hbar

hbar

We now follow the directions in the background for the following derivations.
\begin{align}
&\text{Grating Equation}\quad\ m\lambda\frac{1}{a} = \sin\alpha\pm\sin\beta\\
&\text{Since we use a reflective grating}\quad\ m\lambda\frac{1}{a} = \sin\alpha+\sin\beta\\ 
&\text{Taking the partial derivative with respect to }\lambda\quad\  
\frac{m}{a} = \frac{\partial \beta}{\partial \lambda}\cos\beta\\ 
&\text{Rearranging}\quad\  \frac{m}{a\cos\beta} = \frac{\partial \beta}{\partial \lambda}\\
&\text{Rearranging the grating Equation}\quad\ \lambda = \frac{a}{m}(\sin\alpha+\sin\beta)\\
&\text{Multiplying the above two lines}\quad\  \frac{m}{a\cos\beta}\frac{a}{m}(\sin\alpha+\sin\beta) = \lambda\frac{\partial \beta}{\partial \lambda}\\
&\text{Simplifying the left side}\quad\  \frac{\sin\alpha+\sin\beta}{\cos\beta } = \lambda\frac{\partial \beta}{\partial \lambda}\\
&\text{Since }\alpha -2\theta=\beta \quad\ \frac{\sin(\beta+2\theta)+\sin\beta}{\cos\beta } = \lambda\frac{\partial \beta}{\partial \lambda}\\
&\text{Simplifying and substituting tangent}\quad\ \frac{\sin(\beta+2\theta)}{\cos\beta } + \tan( \beta) = \lambda\frac{\partial \beta}{\partial \lambda}\\
&\text{Since }\sin(a+b)= \sin(a)\cos(b)+\cos(a)\sin(b)\quad\ \frac{\sin(2\theta)\cos(\beta)+\cos(2\theta)\sin(\beta)}{\cos\beta } + \tan( \beta) = \lambda\frac{\partial \beta}{\partial \lambda}\\
&\text{Simplifying and factoring}\quad\  
\sin(2\theta) + \tan( \beta)(1+\cos(2\theta)) = \lambda\frac{\partial \beta}{\partial \lambda}\\ 
&\text{Since }2\cos^2(\theta)-1=\cos(2\theta)\quad\ 
\sin(2\theta) + 2\tan(\beta)\cos^2(\theta) = \lambda\frac{\partial \beta}{\partial \lambda}\\ 
&\blacksquare\nonumber
\end{align}

\begin{align}
&\text{Grating Equation}\quad\ \frac{m\lambda}{a} = \sin(\alpha)+\sin(\beta)\\
&\text{Since }2\theta=\alpha-\beta\quad\ \frac{m\lambda}{a} = \sin(\alpha)+\sin(\alpha-2\theta)\\ 
&\text{Since }2\theta=\alpha-\beta\quad\ \frac{m\lambda}{a} = \sin(\alpha)+\sin(\alpha-2\theta)\\ 
&\text{Sine addition formula}\quad\ \frac{m\lambda}{a} = \sin\alpha(1+\cos(2\theta))-\cos(\alpha)\sin(2\theta)\\
&\text{Trigonometric identity}\quad\ \frac{m\lambda}{a} = 2\sin\alpha\cos^2(\theta))-\cos(\alpha)\sin(2\theta)\\
&\text{Sine addition formula}\quad\ \frac{m\lambda}{a} =  2\sin\alpha\cos^2(\theta))-2\cos(\theta)\cos(\alpha)\sin(\theta)\\
&\text{Rearranging}\quad\ \frac{m\lambda}{2a\cos\theta} =  \sin\alpha\cos(\theta))-\cos(\alpha)\sin(\theta)\\ 
&\text{Sine difference formula}\quad\ \frac{m\lambda}{2a\cos\theta} =
\sin(\alpha-\theta)\\
&\text{Solve for }\alpha\quad\ \arcsin\left(\frac{m\lambda}{2a\cos\theta}\right)+\theta =
\alpha
\end{align}

\begin{align}
    &\text{We know}\quad\ D=i-A+\arcsin\left(n\sin(A-\arcsin\left(\frac{\sin   i}{n}\right)\right)
\end{align}

The desired result can be derived by differentiating both sides with respect to $i$ and then solving for $n.$ Since at the desired $n$, $\frac{dD}{di}=0$ one can use trigonometric identities to algebraically rearrange and solve for $n$. 

In [135]:
data = pd.read_csv("lab4 - The Cooler First Order.csv")
data, end_of_data = data.iloc[:3,:], data.iloc[3:,:]
end_of_data = [x for x in end_of_data.iloc[:,0] if len(str(x))>0
              if type(x) is str]
data

,Slit Width Microns,Separation Between Each Line (mm) on Screen,Rotational Stage Reading When Blazed,Rotational stage reading when the diffraction grating is normal to the beam,Brighter Further Line Separation From Incoming Beam,Dimmer Closer Line Separation From Incoming Beam
0,15 \pm .5,5 mm \pm .5 mm,97 degrees 55 arcminutes \pm 5 arcminutes,95 degrees 35 arcminutes \pm 5 arcminutes,74 \pm .7 mm,64 \pm .7 mm
1,19.5 \pm .5,5 cm \pm 0.5 mm,98 degrees 10 arcminutes \pm 5 arcminutes,95 degrees 25 arcminutes \pm 5 arcminutes,75.5 mm \pm 0.7 mm,70.0 mm \pm 0.7 mm
2,8 \pm 0.5,Not Recorded,97 degrees and 50 arcminutes \pm 5 arcminutes,96 degrees and 5 arcminutes \pm 5 arcminutes,86.0 mm \pm 0.7 mm,76.5 mm \pm 0.7 mm


In [96]:
for note in end_of_data:
    print(note)

distance from diffraction grating to red line on our piece of paper 97.5 \pm 0.7 mm
1200 lines/mm diffraction grating


In [199]:
def convert_data(element):
    try:
        expected_val, uncertainty = [float(x) for x 
            in element.split(" ")
            if len(x)>0 and (x[-1].isnumeric() or x[0].isnumeric())]
        return expected_val, uncertainty
    except: 
        try: 
            expected_val, arcmin, uncertainty = [float(x) for x 
            in element.split(" ")
            if x[-1].isnumeric() or x[0].isnumeric()]
            expected_val = expected_val + arcmin / 60
            return expected_val, uncertainty / 60
        except:
            return "NA"
def arr_to_dict(arr):
    out = {}
    for pair in arr:
        out[pair[0]] = pair[1]
    return out
def convert_col(df, col):
    return arr_to_dict([convert_data(x) for x in data[col]])
slit_widths = convert_col(data, 'Slit Width Microns')
slit_widths

{15.0: 0.5, 19.5: 0.5, 8.0: 0.5}

We now perform an average weighted linearly by the slit width, since it should be proportional to the photon flux on the screen. 

In [200]:
weights = [x for x in slit_widths.keys()]
n = len(weights)

In [201]:
def weighted_average(data, col, weights, coeff=1):
    column = convert_col(data, col)
    keys = coeff * np.array([x for x in column.keys()])
    unc = coeff * [x for x in column.values()][0] / np.sqrt(
            len(weights))
    return { np.sum(np.array([weights[col] * keys[col] for 
            col in range(data.shape[0])])) / np.sum(weights) : unc }
average_angle_blazed = weighted_average(data, 
        'Rotational Stage Reading When Blazed', weights, np.pi/180)
average_angle_blazed

{1.7106964440920107: 0.000839721927886212}

In [202]:
col = 'Rotational stage reading when the diffraction grating is normal to the beam'
average_angle_zeroed = weighted_average(data, col, weights, np.pi/180)
average_angle_zeroed 

{1.668551875977677: 0.000839721927886212}

In [203]:
brighter_further = weighted_average(data, 
    'Brighter Further Line Separation From Incoming Beam', weights)
brighter_further

{76.94705882352942: 0.40414518843273806}

In [204]:
dimmer_closer = weighted_average(data, 
    'Dimmer Closer Line Separation From Incoming Beam', weights)
dimmer_closer

{69.10588235294118: 0.40414518843273806}

I have written a publicly available error propagation Python script on Github, which I make use of here. The purpose of this file is to have a set of functions that can calculate error propagation.
All inputs will be dictionaries of the form

dict = {value1: uncertainty1, value2: uncertainty2, ...}

All outputs will be lists of the form (value, uncertainty).
All uncertanties should be listed as nonnegative real numbers.
For the general() function to calculate the exact uncertainty,
the syntax for defining functions used here:

https://www.askpython.com/python/examples/derivatives-in-python-sympy

should be used.
The purpose of symbol_list list is to have up to 26 variable names to take
partial differentials in general()
All functions used in the general function must use the first n letters of
the alphabet as their variables

In [205]:
a, b, c, d, e, f, g, h, i, j, k, l, m = symbols('a b c d e f g h i j k l m')
n, o, p, q, r, s, t, u, v, w, x, y, z = symbols('n o p q r s t u v w x y z')
symbol_list = (a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v,
w, x, y, z)

def general (dict, fun):
    #can only take up to 26 inputs
    #its output is of the form (exact uncertainty, maximum uncertainty)
    sum_squares = 0

    try:
        keys = np.array(list(dict.keys()))
        values = np.array(list(dict.values()))
        for index in range(len(keys)):
            diff_f = fun.diff(symbol_list[index])
            lam_f = lambdify(symbol_list[0:len(keys)], diff_f)
            sum_squares += (lam_f(*keys) * values[index]) ** 2
        lam_og = lambdify(symbol_list[0:len(keys)], fun)
        best_est = lam_og(*keys)
    except:
        print("Singular uncertainty entered")
        keys = list(dict.keys())
        keys = float(keys[0])
        values = list(dict.values())
        values = float(values[0])
        diff_f = fun.diff(symbol_list[0])
        lam_f = lambdify(symbol_list[0], diff_f)
        sum_squares += (lam_f(keys) * values) ** 2
        lam_og = lambdify(symbol_list[0], fun)
        lam_og(keys)

    uncertainty = math.sqrt(sum_squares)

    return (best_est, uncertainty)
#a = x, y = b, c = theta
fun = (a + 2) / (a + (b * cos(4 * c * np.pi / 180)))
dict = {10:2, 7:1, 40:3}
print(general(dict, fun))

(3.50656581341894, 1.8267621151828408)


a = l1, b = linear separations (brigth lines)

c is the starting angle, d is the grating angle

e is $\alpha$ and will be a function of the c and d. 

f is $\beta$ and will be a function of e, a, and b. 

g is $\lambda$ will be a function of e and f.

In [206]:
arctan = atan
angle_when_blazed, angle_when_zeroed = symbols("a b")
d_n, grating_to_red_line = symbols("c d")
alpha = angle_when_blazed - angle_when_zeroed
beta = arctan(d_n / grating_to_red_line) - alpha
wavelength = 10**6 * (1/1200) * (sin(alpha) + sin(beta))
md("Where atan is arctan, a is the angle on rotional stage, b" 
   " is the angle on the rotational stage when normal to the " 
   "incoming beam, d is the distance from the diffraction grating"
   " to an arbitrary line normal to the incoming beam," +  
   "c is the distance from the spectral feature on the screen" + 
    "from the incoming beam along the same arbitrary line" + 
   " we have for the wavelength in nanometers\\begin{equation}" + 
   "\\lambda=" + latp(simplify(wavelength)) +"\\end{equation}")

Where atan is arctan, a is the angle on rotional stage, b is the angle on the rotational stage when normal to the incoming beam, d is the distance from the diffraction grating to an arbitrary line normal to the incoming beam,c is the distance from the spectral feature on the screenfrom the incoming beam along the same arbitrary line we have for the wavelength in nanometers\begin{equation}\lambda=833.333333333333 \sin{\left(a - b \right)} + 833.333333333333 \sin{\left(- a + b + \operatorname{atan}{\left(\frac{c}{d} \right)} \right)}\end{equation}

In [234]:
averaged_dict = (average_angle_zeroed | average_angle_blazed |
            brighter_further | {97.5:.7} )
lam_brighter_further = general(averaged_dict, wavelength)
md("We have for the wavelength of the brighter emission line" + 
   "in the doublet in nm" +  "\\begin{equation}\\Large\\lambda=" + 
   latp(lam_brighter_further[0]) + "\\pm" + 
   latp(lam_brighter_further[1]) + "\\end{equation}")

We have for the wavelength of the brighter emission linein the doublet in nm\begin{equation}\Large\lambda=508.252648723968\pm2.74364673383703\end{equation}

In [224]:
averaged_dict = (average_angle_zeroed | average_angle_blazed |
            dimmer_closer | {97.5:.7} )
lam_dimmer_closer = general(averaged_dict, wavelength)
md("We have for the wavelength of the brighter emission line in the" +
   " doublet in nm\\begin{equation}\\Large\\lambda=" + 
   latp(lam_dimmer_closer[0]) + 
   "\\pm" + latp(lam_dimmer_closer[1]) + "\\end{equation}")

We have for the wavelength of the brighter emission line in the doublet in nm\begin{equation}\Large\lambda=474.989240424089\pm2.88615877888694\end{equation}

In [237]:
e = d - c  
f = arctan( b / a) - e
g = 10**-3 / (1200) * (sin(e) + sin(f))
b_arr = [74.0, 75.5, 86.0]
c_arr = np.pi * np.array([95+35/60, 95 + 25/60, 96 + 5/60]) / 180
d_arr = np.pi * np.array([97+55/60, 98 + 10/60, 97 + 50/60]) / 180
angle_unc = 5 * np.pi / ( 60 * 180 )
output = [general({97.5:.7, b_arr[i]:.7, c_arr[i]:angle_unc, 
                d_arr[i]:angle_unc}, g) for i in range(len(b_arr))]
md("We have for the wavelengths and uncertainties of the doublet"
   "\\begin{equation}" + latp(output) + "\\end{equation}"+ 
   "and for the width of each slit and their uncertainty in microns, respectively" + 
   "\\begin{equation}" + str(slit_widths) + "\\end{equation}")

We have for the wavelengths and uncertainties of the doublet\begin{equation}\left[ \left( 5.1028905792447 \cdot 10^{-7}, \  3.92228677149986 \cdot 10^{-9}\right), \  \left( 5.17994675481773 \cdot 10^{-7}, \  3.88705235226923 \cdot 10^{-9}\right), \  \left( 5.57351711971439 \cdot 10^{-7}, \  3.47638104232865 \cdot 10^{-9}\right)\right]\end{equation}and for the width of each slit and their uncertainty in microns, respectively\begin{equation}{15.0: 0.5, 19.5: 0.5, 8.0: 0.5}\end{equation}

# Discussion
As expected, the spectral resolution somewhat increased with smaller slit widths, but the doublet reflected by the diffraction grating onto the screen was noticeably dimmer. Although a smaller slit width can allow for a higher spectral resolution via the grating equation, it can also act as a field stop and decrease the flux of light onto the screen. For instance, if one were using a CCD instead of a screen, then for a dim light source then the total photon counts would decrease, potentially impacting the science goals of the experiment or observation. Thus, a smaller slit width is not always monotonically better, and one must carefully weigh the trade-off between photon throughput and spectral resolution. For instance, a slit is too thin may create unnecessary uncertainty in the depth of a spectral feature that is of interest to the investigation. 

The focal lengths of collimating biconvex lenses were not particularly significant because as long as an image was formed on the screen, then changing the distance of the screen would simply make the image larger, smaller, or out of focus. Any damage or aberrations in any of the components in the optical train, or misallignment in the vertical plane would be detrimental to the spectral resolution. In practice, systematic errors and noise due to the process with which we measured each distance used in the wavelength calculation were easily the largest sources of error. 

Similar findings would be expected for the zeroeth order, but of course without spectral resolution playing a role since the zeroeth order cannot resolve spectral features. 

Were part 2 of the experiment possible without time constraints, the values measured would be substituted into the Equation for spectral resolution derived above in this report, and it would be expected to be smaller than the same value of diffraction grating spectrographs. 

In [225]:
md("If we instead seek to find a singular wavelength for the entire"
  + " doublet then we have in meters\\begin{equation}\\large\\lambda=" + 
  latp(np.average([x[0] for x in output])) + "\\pm" + 
   latp(np.average([x[1] for x in output]/np.sqrt(len(output)))) + 
  "\\end{equation}") 

If we instead seek to find a singular wavelength for the entire doublet then we have in nm\begin{equation}\large\lambda=5.28545148459227 \cdot 10^{-7}\pm2.17193785863177 \cdot 10^{-9}\end{equation}

Each member of the lab group contributed equally, since we implemented a system where after each set of measurements, we would rotate who was participating in what role. 

In [240]:
!jupyter nbconvert --to latex Lab4.ipynb

[NbConvertApp] Converting notebook Lab4.ipynb to latex
[NbConvertApp] Writing 60519 bytes to Lab4.tex
